In [76]:
import gensim
import pickle
from nltk.stem import WordNetLemmatizer
import json
import re
from nltk.tokenize import word_tokenize
from nltk.sentiment.util import mark_negation
import nltk
import os
import utils
from random import shuffle


## Constants

In [77]:
eval_data_file='data/eval_data.txt'
eval_words_file='data/eval_data.txt.td'


## Read Data

In [78]:
reader=open(eval_words_file,'r')
eval_words_list=[]
for line in reader:
    tok_line=line.strip().split(" ")
    x=len(tok_line)
    for i in range(x):
        tok_line[i]=tok_line[i].split(":")[-1]
    eval_words_list.append(tok_line)
print(len(eval_words_list))
reader.close()

21918


In [82]:
reader=open(eval_data_file,'r')
context_list=[]
y=[]
# context_len=3
for line in reader:
    tok_line=line.strip().split("<<target>>")
    context=gensim.utils.simple_preprocess(tok_line[0])
    x=len(context)
#     if(x>context_len):
#         context=context[x-context_len:x]
    
    y.append(tok_line[1].split(":")[-1])
    context2=gensim.utils.simple_preprocess(tok_line[1])

    if len(context2)>0:
        context2.pop()
#         if len(context2)>context_len:
#             context2=context2[0:context_len]
    context_list.append(context)
    context_list.append(context2)
print(len(context_list),len(y))
print(context_list[0],y[0])
print(context_list[1],y[0])

43836 21918
['the', 'particulars', 'of', 'your', 'plans', 'neither'] know
['or', 'seek', 'to', 'know'] know


## Load Model

In [80]:
def write_file(arr,file):
    with open(file,'w') as f:
        for line in arr:
            st=" ".join([str(x) for x in line])
            print(st,file=f)
    f.close()

In [85]:
def func(func,save_path,flag=False,hits=2000,context_win=2):

    mrr=0
    mr=0
    ranking=[]
    upto=2000

    if flag==True:
        upto=len(eval_words_list)

    for i in range(upto):
        if i%100==0:
            print(mrr/(i+1),mr*1.0/(i+1),i)
        context1=context_list[2*i]
        context2=context_list[2*i+1]
        x1=len(context1)
        x2=len(context2)
        if x1>context_win:
            context1=context1[x1-context_win:x1]
        if x2>context_win:
            context2=context2[0:context_win]
        
        context=context1+context2
        word=y[i]
        val=func(context,topn=hits)
        local_rank=[]
        mapper={}
        mapper_count={}
        for w in eval_words_list[i]:
            mapper[w]=0
            if w not in mapper_count:
                mapper_count[w]=0
            mapper_count[w]+=1

        count=1
        if val is not None:
            for j in range(len(val)):
                if val[j][0] in mapper:
                    mapper[val[j][0]]=count
                    count+=mapper_count[val[j][0]]

        temp=[]
        for w in eval_words_list[i]:
            if mapper[w]==0:
                temp.append(w)
        shuffle(temp)
        
        for w in temp:
            if mapper[w]==0:
                mapper[w]=count
                count+=mapper_count[w]

        for w in eval_words_list[i]:
            local_rank.append(mapper[w])
            mapper[w]+=1
        ranking.append(local_rank)

        if word not in mapper:
            print (word)
            print (eval_words_list[i])
            print (mapper)
        
        mr+=(mapper[word]-1)
        mrr+=1.0/(mapper[word]-1)
    write_file(ranking,save_path)
    return ranking

In [161]:
model_path='./model.pkl'
# model=utils.load_pickle(model_path)

In [162]:
model_path2='./model_new.pkl'
# model2=utils.load_pickle(model_path2)

In [164]:
model_path3='./model_pret_upd.pkl'
model3=utils.load_pickle(model_path3)

In [96]:
%time t=func(model4.predict_output_word,'out.txt',flag=False,hits=2000,context_win=1)

0.0 0.0 0
0.30699876847587376 34.35643564356435 100
0.2592272897925406 38.59701492537314 200
0.23116127124633282 40.970099667774086 300
0.2238166199876675 40.48628428927681 400
0.21601392892884974 41.275449101796404 500
0.222019537821265 41.69051580698835 600
0.21835305226342266 40.639087018544934 700
0.21720847686204048 42.2334581772784 800
0.22084885012517555 41.62486126526082 900
0.22524608331121712 40.74625374625375 1000
0.22513302884448103 40.91099000908265 1100
0.22664696601334341 40.72439633638634 1200
0.22897009093245071 40.34050730207532 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.2298955533781944 40.835831548893644 1400
0.23157222774862904 40.45836109260493 1500
0.2313354948413428 40.64522173641474 1600
0.23522962276587495 40.44326866549089 1700
0.2356630633484433 40.671848972792894 1800
0.23841770362644724 40.69331930562862 1900
CPU times: user 5min 26s, sys: 1min 17s, total: 6min 43s
Wall time: 32.6 s


In [97]:
model_path5='./fresh_emb50_ep10.pkl'
model5=utils.load_pickle(model_path5)
%time t=func(model5.predict_output_word,'out.txt',flag=False,hits=2000,context_win=1)

0.0 0.0 0
0.27556806013898033 40.91089108910891 100
0.2338042664466532 46.37313432835821 200
0.2170760483157884 44.970099667774086 300
0.21218504807213362 46.16957605985037 400
0.20508144103122128 46.455089820359284 500
0.21097634560908698 45.54742096505824 600
0.21239203798714998 43.7018544935806 700
0.21368437041977417 43.90387016229713 800
0.21878252276343224 42.900110987791344 900
0.2241632114235061 41.86113886113886 1000
0.22304672167630593 42.653950953678475 1100
0.22606150521021662 42.97418817651957 1200
0.2293945584553593 42.97924673328209 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.23050976662097314 43.54032833690221 1400
0.23227407577094206 43.17388407728181 1500
0.23052078014370825 43.38226108682074 1600
0.23355605613348715 43.24867724867725 1700
0.23487250043810437 43.217101610216545 1800
0.23741006019639085 43.20778537611783 1900
CPU times: user 5min 50s, sys: 1min 21s, total: 7min 11s
Wall time: 34.5 s


In [100]:
model_path6='./fresh_emb50_ep40.pkl'
model6=utils.load_pickle(model_path6)
%time t=func(model6.predict_output_word,'out.txt',flag=False,hits=2000,context_win=1)

0.0 0.0 0
0.21754118988649676 46.45544554455446 100
0.19308495165404643 51.46268656716418 200
0.17500531595573474 51.57475083056478 300
0.17322406595636386 53.33167082294264 400
0.1723770322803955 54.30538922155689 500
0.17863225758783977 52.64226289517471 600
0.17780970326025924 51.991440798858775 700
0.17717150238953489 52.61922596754057 800
0.18144170237369261 51.62930077691454 900
0.18267458066138806 49.91908091908092 1000
0.1820768265271428 49.79745685740236 1100
0.1831914792034789 50.07910074937552 1200
0.18604672901702216 49.50730207532667 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.18704995754774326 49.75017844396859 1400
0.19014757130628526 49.51165889407062 1500
0.18793672225053665 49.72517176764522 1600
0.18990275621792918 49.46325690770135 1700
0.18961486141962597 49.33037201554692 1800
0.19234056995177318 49.556023145712786 1900
CPU times: user 5min 54s, sys: 1min 25s, total: 7min 19s
Wall time: 35.8 s


In [101]:
model_path7='./pretr_emb300_ep40.pkl'
model7=utils.load_pickle(model_path7)
%time t=func(model7.predict_output_word,'out.txt',flag=False,hits=2000,context_win=1)

0.0 0.0 0
0.2659592940020254 45.7029702970297 100
0.22100156218280334 45.62189054726368 200
0.20272434553887314 48.863787375415285 300
0.20613993400759956 51.32668329177057 400
0.20076824749493602 52.21157684630739 500
0.21048067561360864 49.85191347753744 600
0.20158334759810628 49.798858773181166 700
0.20082641154617184 49.66541822721598 800
0.2070187566622891 48.35294117647059 900
0.20819243973792856 47.41558441558441 1000
0.20816622032796328 47.25522252497729 1100
0.21100180887938347 47.59866777685262 1200
0.2133385474432966 47.49500384319754 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.21386012759953243 48.29835831548893 1400
0.21837802376078239 48.00732844770153 1500
0.21515853299078647 48.2792004996877 1600
0.21652358705829106 48.145796590241034 1700
0.21567568641788215 48.38922820655191 1800
0.21690116197509793 48.664913203577065 1900
CPU times: user 5min 54s, sys: 1min 19s, total: 7min 14s
Wall time: 34.8 s


In [106]:
model_path7='./pretr_emb100_ep10.pkl'
model7=utils.load_pickle(model_path7)
%time t=func(model7.predict_output_word,'out.txt',flag=False,hits=2000,context_win=1)

0.0 0.0 0
0.2964235709059789 28.524752475247524 100
0.24998609682022357 36.01990049751244 200
0.23300993099469675 40.40199335548173 300
0.22471921265620734 43.38902743142145 400
0.21806156678522232 43.94411177644711 500
0.22554977955798083 43.29450915141431 600
0.22518059536907187 41.96005706134094 700
0.22299569847461675 42.729088639200995 800
0.22729888544185078 43.04328523862375 900
0.22917329739335066 41.968031968031966 1000
0.22953115129569254 42.39600363306085 1100
0.23495138888515274 42.255620316403 1200
0.23758162372107583 42.42659492697925 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.23928789331368244 42.40899357601713 1400
0.24033210991243487 42.02998001332445 1500
0.23964079508394526 41.91942535915053 1600
0.24242520008603163 41.63609641387419 1700
0.2420668165826302 41.67351471404775 1800
0.24331359657171445 41.6449237243556 1900
CPU times: user 5min 26s, sys: 1min 16s, total: 6min 43s
Wall time: 32.7 s


In [107]:
model_path7='./pretr_emb100_ep20.pkl'
model7=utils.load_pickle(model_path7)
%time t=func(model7.predict_output_word,'out.txt',flag=False,hits=2000,context_win=1)

0.0 0.0 0
0.25743956926135403 33.68316831683168 100
0.21476457362557208 41.776119402985074 200
0.1965116743038974 46.19601328903654 300
0.197873268610301 47.556109725685786 400
0.1968332947746583 48.62674650698603 500
0.20575075992612488 48.70049916805324 600
0.20015850259916315 47.574893009985736 700
0.20043057132462927 48.24219725343321 800
0.2011804663775172 47.69256381798002 900
0.20268926380520708 46.63036963036963 1000
0.20461120871166352 46.3133514986376 1100
0.2052723558231145 46.62531223980017 1200
0.20772419426606364 46.23750960799385 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.2111194051383877 45.92291220556745 1400
0.21378160558535447 46.07128580946036 1500
0.2121401187886767 46.59712679575266 1600
0.2144046459117761 46.08818342151675 1700
0.2145792562558492 46.2870627429206 1800
0.21707269070961174 46.011572856391375 1900
CPU times: user 6min 5s, sys: 1min 22s, total: 7min 27s
Wall time: 34.3 s


In [108]:
model_path7='./pretr_emb100_ep5.pkl'
model7=utils.load_pickle(model_path7)
%time t=func(model7.predict_output_word,'out.txt',flag=False,hits=2000,context_win=1)

0.0 0.0 0
0.2604820060015383 35.277227722772274 100
0.23704356762582343 41.38308457711443 200
0.22127233906737503 44.9734219269103 300
0.21161229256977823 46.78304239401496 400
0.20260261117404627 47.40119760479042 500
0.2059363386843154 46.32778702163061 600
0.20658835135773668 46.02853067047076 700
0.20512363527511923 46.58302122347066 800
0.20892259393121043 46.126526082130965 900
0.21155368151955875 45.85114885114885 1000
0.2134912152797224 45.97820163487739 1100
0.21871808900119674 45.77601998334721 1200
0.2225250884197275 45.446579554189086 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.22343756210557855 45.45895788722341 1400
0.22667850731757608 45.52098600932712 1500
0.22667032252829256 45.65209244222361 1600
0.23120129838839676 44.78130511463845 1700
0.2339452564880027 44.389783453636866 1800
0.23456826238575265 44.52551288795371 1900
CPU times: user 5min 49s, sys: 1min 21s, total: 7min 10s
Wall time: 34.6 s


In [115]:
model_path7='./pretr_emb150_ep10.pkl'
model7=utils.load_pickle(model_path7)
%time t=func(model7.predict_output_word,'out.txt',flag=False,hits=2000,context_win=1)

0.0 0.0 0
0.28416679587008953 31.613861386138613 100
0.24111261706751305 36.87064676616915 200
0.22295721765321205 42.538205980066444 300
0.21789607905636216 45.18204488778055 400
0.21267836956069985 45.387225548902194 500
0.222749358824198 44.22129783693843 600
0.22345533208489055 42.70898716119829 700
0.2235623645863348 43.49438202247191 800
0.22672566871629188 42.94672586015538 900
0.2280609090153869 42.285714285714285 1000
0.226665921443962 42.31607629427793 1100
0.23178835098123426 42.22564529558701 1200
0.23537815404257068 42.2782475019216 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.2373430928733352 42.43397573162027 1400
0.24061315091114266 42.29113924050633 1500
0.2386698728141829 42.30980637101811 1600
0.2416889381183515 42.03233392122281 1700
0.24165531318126618 41.94058856191005 1800
0.24346127438501766 41.86533403471857 1900
CPU times: user 5min 38s, sys: 1min 21s, total: 7min
Wall time: 34.5 s


In [120]:
model_path7='./sg_emb150_ep10.pkl'
model7=utils.load_pickle(model_path7)
%time t=func(model7.predict_output_word,'out.txt',flag=True,hits=2000,context_win=1)

0.0 0.0 0
0.27446284865940307 36.13861386138614 100
0.23601377360468762 35.52238805970149 200
0.22097167759649608 34.7109634551495 300
0.218521627824526 36.87780548628429 400
0.21400618900627086 38.003992015968066 500
0.2188523178828664 37.58901830282862 600
0.21786107386722936 36.78174037089872 700
0.21833414931431996 37.45942571785268 800
0.22449575670458707 36.82019977802442 900
0.2256743739470909 36.064935064935064 1000
0.22646287841848842 36.32970027247956 1100
0.22768131766801014 36.02581182348043 1200
0.2330668162635232 35.38816295157571 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.2335661554235613 35.47180585296217 1400
0.23792061587414431 35.241838774150565 1500
0.2376005336944107 35.01748906933167 1600
0.23938994510293607 34.56025867136978 1700
0.2406314616909294 34.61743475846752 1800
0.2412270786405027 34.71330878485008 1900
0.24429966306731019 34.3608195902049 2000
0.24326383443267974 34.254164683484056 2100
0.24055618325229078 34.695138573375736 2200
0.24394686503349128 34.61712299000435 2300
0.24385453154679512 34.7076218242399 2400
0.24643627015091682 34.63014794082367 2500
0.2461681178713544 34.855055747789315 2600
0.24403716424722277 35.242502776749355 2700
0.2423612649499035 35.661906461977864 2800
0.2410907092201169 36.00068941744226 2900
0.23915032972156566 36.080639786737756 3000
0.23734971644172148 36.330538535956144 3100
0.23783448895610837 36.51265229615745 3200
0.2375472317225078 36.409572856710085 3300
0.2379631519836678 36.14260511614231 3400
0.2379382580301062 36.01713796058269 3500
0.23731422421651727 36.03860038878089 3600
0.236539773745

0.26660070807301917 31.32611146512426 20400
0.2671664234507103 31.288083508121556 20500
0.26732223413211054 31.28304451240231 20600
0.26714948237453673 31.311675764455824 20700
0.26714049896356945 31.29633190711985 20800
0.26731465619999767 31.27357542701306 20900
0.26742359019255585 31.2827008237703 21000
0.2676013147031084 31.262594189848823 21100
0.2676447231548177 31.27272298476487 21200
0.2678875531441963 31.25102107882259 21300
0.26807239466874766 31.239241156955284 21400
0.2680841250314184 31.274266313194737 21500
0.268289266895464 31.24026665432156 21600
0.2684021906892279 31.220727155430627 21700
0.268539098375469 31.247190495848816 21800
0.26862085833681076 31.248710104561436 21900
CPU times: user 1h 3min 35s, sys: 14min 44s, total: 1h 18min 19s
Wall time: 6min 6s


In [119]:
model_path7='./sg_pret_emb300_ep10.pkl'
model7=utils.load_pickle(model_path7)
%time t=func(model7.predict_output_word,'out.txt',flag=True,hits=2000,context_win=1)

0.0 0.0 0
0.2742746540331958 31.06930693069307 100
0.2369225808863867 31.691542288557216 200
0.22017377427219534 36.920265780730894 300
0.22195558563474502 36.25935162094763 400
0.21983834541497085 36.87624750499002 500
0.22405396143287154 35.871880199667224 600
0.21879323727037883 35.1811697574893 700
0.21981983070969566 36.34956304619226 800
0.22648215173407146 35.21198668146504 900
0.22964577749580778 34.51048951048951 1000
0.2323178006258573 35.071752951861946 1100
0.2316012309635067 34.945878434637805 1200
0.2372386018264149 34.427363566487315 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.23943496985335294 34.54675231977159 1400
0.2440851667242745 34.5709526982012 1500
0.24037283553286978 35.00124921923798 1600
0.24229826192975082 34.63609641387419 1700
0.24263836176925177 34.75013881177124 1800
0.24235578668748406 34.850078905839034 1900
0.2427206249405747 34.640679660169916 2000
0.24055345470110037 34.85625892432175 2100
0.2370589071289382 35.12448886869605 2200
0.2405292961395843 34.850499782703174 2300
0.24034336577989987 34.71012078300708 2400
0.24420821524382472 34.358256697321075 2500
0.24330244891011213 34.69127258746636 2600
0.24162990276907917 34.79192891521659 2700
0.24011502988897257 34.982506247768654 2800
0.23952730817557705 35.10410203378145 2900
0.23771092584586292 34.973008997001 3000
0.2367900246959174 35.34182521767172 3100
0.23698131377102796 35.466104342393 3200
0.23809924033280777 35.379884883368675 3300
0.23903885971489156 35.33695971773008 3400
0.23999162851891961 35.14224507283633 3500
0.23910390636185208 35.226048319911136 3600
0.238606620894

0.2648644819223565 30.220481749667503 20300
0.26470326104309133 30.240821528356452 20400
0.26525160225182526 30.184673918345446 20500
0.2655516814564834 30.164700742682395 20600
0.2654043093278353 30.154871745326314 20700
0.2653225175672784 30.184798807749626 20800
0.2655061296471316 30.152911343954834 20900
0.2655797056290603 30.14789771915623 21000
0.26564134401287615 30.11184304061419 21100
0.26568143380472287 30.10513655016273 21200
0.2658536009915869 30.09717853621896 21300
0.26610490154836036 30.074529227606188 21400
0.26602941783634776 30.124784893725874 21500
0.2663230549189751 30.1164297949169 21600
0.2663705584038669 30.124556472051978 21700
0.2665507085429388 30.130223384248428 21800
0.2667076210311833 30.127163143235467 21900
CPU times: user 1h 7min 58s, sys: 15min 5s, total: 1h 23min 4s
Wall time: 6min 25s


## Fasttext

In [69]:
model_path8='./fsttxt_emb100_ep10.pkl'
model8=utils.load_pickle(model_path8)
%time t=func(model8.wv.most_similar)

0.0 0.0 0
0.02460274611487706 89.77227722772277 100
0.03220593072892044 89.4776119402985 200
0.029540804434379363 87.70431893687707 300
0.027192594103889866 88.7930174563591 400
0.025392469008578247 90.16966067864271 500
0.026176918822589264 92.9783693843594 600
0.025776373758194318 91.71469329529243 700
0.026208694635376794 91.4494382022472 800
0.027231475378486893 90.60266370699223 900
0.0270994536157691 90.43856143856144 1000
0.0272669873569721 89.88283378746594 1100
0.028605883696238555 89.4029975020816 1200
0.02822793113599507 89.11683320522675 1300
0.027791460378587186 89.29122055674519 1400
0.027586324285151505 89.69820119920054 1500
0.027659865226035397 89.63897564022486 1600
0.02883287566802623 89.33509700176367 1700
0.02824987640754266 89.4764019988895 1800
0.02805389708111949 89.3550762756444 1900
CPU times: user 3min 14s, sys: 43.9 s, total: 3min 58s
Wall time: 17.6 s


In [74]:
print(model8.wv.most_similar(['How','about','going','and','eating'],topn=10))
print(model6.predict_output_word(['How','about','going','and','eating'],topn=10))

[('tang', 0.9311258792877197), ('retching', 0.9282938241958618), ('reading', 0.9261969327926636), ('retaking', 0.9244906902313232), ('wrang', 0.9223257303237915), ('wreaking', 0.9219764471054077), ('agog', 0.9214711785316467), ('replying', 0.918215811252594), ('reckoning', 0.918073296546936), ('dreaming', 0.9177627563476562)]
[('worry', 0.0034242703), ('deal', 0.0016757236), ('wheat', 0.0013753988), ('weeks', 0.0012984276), ('row', 0.0012913616), ('annie', 0.0011960537), ('pins', 0.00092837994), ('sell', 0.00092607184), ('consult', 0.00086540275), ('wedding', 0.0008534614)]


In [213]:
a="::::toils"
a.split(":")

['', '', '', '', 'toils']

In [216]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sent_detector.tokenize("I am Ankesh ::Gupta. I am four years old")

['I am Ankesh ::Gupta.', 'I am four years old']

In [19]:
print(gensim.utils.simple_preprocess("I doubt whether since I saw you , you have done <<target>> good whole day 's work , in any one day .::::a"))
tt="I doubt whether since I saw you , you have done <<target>> good whole day 's work , in any one day .::::a"
tt="This habit of uselessly wasting time , is the whole difficulty ; and it is vastly important <<target>> you , and still more so to your children , that you should break this habit .::::to"
gensim.utils.simple_preprocess((tt.split("<<target>>"))[1])

['you',
 'and',
 'still',
 'more',
 'so',
 'to',
 'your',
 'children',
 'that',
 'you',
 'should',
 'break',
 'this',
 'habit',
 'to']